# load data

In [1]:
from data import store, info
store

<class 'pandas.io.pytables.HDFStore'>
File path: ./_data/store.db
/ad                            frame        (shape->[6582,6])    
/app_categories                frame        (shape->[217041,2])  
/feat-ad-default               frame        (shape->[6582,9])    
/feat-user-default             frame        (shape->[2805118,12])
/position                      frame        (shape->[7645,3])    
/test                          frame        (shape->[338489,8])  
/train                         frame        (shape->[3749528,8]) 
/user                          frame        (shape->[2805118,8]) 
/user_app_actions              frame        (shape->[6003471,3]) 
/user_installedapps            frame        (shape->[84039009,2])

In [2]:
train=store['train']
train.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,0,170000,NaN,3089,2798058,293,1,1
1,0,170000,NaN,1259,463234,6161,1,2
2,0,170000,NaN,4465,1857485,7434,4,1
3,0,170000,NaN,1004,2038823,977,1,1
4,0,170000,NaN,1887,2015141,3688,1,1


# load raw features

In [3]:
f_user = store['feat-user-default']
f_user.head()

,userID,age,gender,education,marriageStatus,haveBaby,hometown,residence,appCate1_act,count_act,appCate1_inst,count_inst
0,1,42,1,0,2,0,512,503,1,1,0,79
1,2,18,1,5,1,0,1403,1403,NaN,NaN,NaN,NaN
2,3,0,2,4,0,0,0,0,NaN,NaN,NaN,NaN
3,4,21,2,5,3,0,607,607,NaN,NaN,NaN,NaN
4,5,22,2,0,0,0,0,1301,1,9,NaN,NaN


In [4]:
f_ad = store['feat-ad-default']
f_ad.head()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform,appCate2,appCate1,appCategory
0,4079,2318,147,80,14,2,2,0,2
1,4565,3593,632,3,465,1,9,2,209
2,3170,1593,205,54,389,1,8,1,108
3,6566,2390,205,54,389,1,8,1,108
4,5187,411,564,3,465,1,9,2,209


In [5]:
f_pos = store['position']
f_pos.head()

,positionID,sitesetID,positionType
0,2150,1,0
1,2579,1,0
2,3322,1,0
3,5726,1,0
4,4522,2,0


# experiment by your self to design a extractor 

In [6]:
train = store['train']
test = store['test']
train_feats = train.columns
test_feats = test.columns
label_name = 'label'
print train_feats
print test_feats
print label_name

Index([u'label', u'clickTime', u'conversionTime', u'creativeID', u'userID',
       u'positionID', u'connectionType', u'telecomsOperator'],
      dtype='object')
Index([u'instanceID', u'label', u'clickTime', u'creativeID', u'userID',
       u'positionID', u'connectionType', u'telecomsOperator'],
      dtype='object')
label


In [7]:
common_feat = list(set(train_feats).intersection(set(test_feats)))
common_feat.remove(label_name)
print common_feat

class RawData(object):
    pass
raw_data = RawData()

raw_data.store = store
raw_data.label_name = label_name
raw_data.common_feat = common_feat

['positionID', 'clickTime', 'userID', 'connectionType', 'creativeID', 'telecomsOperator']


In [8]:
#%%writefile competition/featExtract/simple_features.py

from competition.featExtract.base import Extractor
import pandas as pd



class SimpleFeatures(Extractor):
    def __init__(self, raw_data):
        self.raw_data = raw_data
        store = raw_data.store
        f_user = store['feat-user-default']
        f_ad = store['feat-ad-default']
        f_pos = store['position']
        def merge_tables(X):
            # user
            X = pd.merge(X,f_user,how='left',left_on='userID',right_on='userID')
            del X['userID']
            # ad
            X = pd.merge(X,f_ad,how='left',left_on='creativeID',right_on='creativeID')
            del X['creativeID']
            # pos
            X = pd.merge(X,f_pos,how='left',left_on='positionID',right_on='positionID')
            del X['positionID']

            return X
        self.merge_tables = merge_tables

    def get_train(self):
        X,y = self._get(self.raw_data.store['train']) 
        X = self.merge_tables(X)
        return X,y

    def get_test(self,dataframe):
        X,y = self._get(self.raw_data.store['test']) 
        X = self.merge_tables(X)
        return X,y

    def _get(self,dataframe):
        X = dataframe[self.raw_data.common_feat]
        y = dataframe[self.raw_data.label_name]
        return X,y


In [9]:
from competition.featExtract.simple_features import SimpleFeatures
prep = SimpleFeatures(raw_data)

trainX,trainY = prep.get_train()
testX,testY  = prep.get_train()


In [10]:
print trainX.shape
print testX.shape

(3749528, 24)
(3749528, 24)


# run the extractor
- modify `config.py`
- run `extract_features.py`

In [11]:
!cat config.py

# -*- coding:utf-8 -*-

# define at you will
# this file is intended to be used by the scripts in this folder


# convert_data, load_data
dataDir = './_data/'
extractDir = './_extracted/'
resultDir = './_result/'

dataFile = 'store.db'
infoFile = 'info.pkl'

# extract_features
from competition.featExtract.simple_features import SimpleFeatures as Extractor
Extractor_name = Extractor.__module__.split('.')[-1]



In [12]:
%run extract_features.py

In [13]:
!ls ./_extracted/

simple_features.db
